In [ ]:
from herbie import Herbie
#from toolbox import EasyMap, pc
#from toolbox.mapping import EasyMap
#from paint.standard2 import cm_tmp

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import traceback  # Add this line to import the traceback module

# https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/ex_python_plot_zarr.html

import s3fs
import numcodecs as ncd
import numpy as np
import datetime
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import geopandas as gpd
import pandas as pd
import datetime
import math
from shapely.ops import unary_union
from shapely.geometry import Point, LineString
import geemap
import numpy as np
from datetime import timedelta
import time

# Open the shapefile and create a GeoDataFrame
#gdf = gpd.read_file("C:\\Users\\magst\\Desktop\\MODISVIIRS\\data_for_jessie_young\\all_perimeters.shp")
#gdf = gpd.read_file("C:\\Users\\magst\\Desktop\\MODISVIIRS\\data_for_jessie_young\\perimeters_NorCal_2021.shp")
#gdf = gpd.read_file("C:\\Users\\magst\\Desktop\\MODISVIIRS\\data_for_jessie_young\\perimeters_NorCal_2019_2022.shp")
#gdf = gpd.read_file("C:\\Users\\magst\\Desktop\\MODISVIIRS\\VIIRS_Perimeters\\combined_perimeter_19.shp")
#gdf = gpd.read_file("C:\\Users\\magst\\Desktop\\MODISVIIRS\\data_for_jessie_young\\perimeters_NorCal_2019_2022.shp")
#gdf = gpd.read_file("C:\\Users\\magst\\Desktop\\MODISVIIRS\\VIIRS_Perimeters\\combined_perimeter_19_20_21.shp")


gdf = gpd.read_file("C:\\Users\\admin-magstadt\\Desktop\\GRIDMET_VIIRS_10_9_23\\combined_perimeter_19_20_21.shp")

# Convert the timestamp column to datetime type
gdf['t'] = pd.to_datetime(gdf['t'])

gdf = gdf[gdf['t'].dt.year == 2021]

# Group the GeoDataFrame by 't' and 'FireName' and apply unary_union to combine the polygons
#grouped = gdf.groupby(['t', 'FireName'])
grouped = gdf.groupby(['t', 'fireid'])
#grouped = gdf.groupby(['t', 'FireID'])

# Create a new GeoDataFrame with the combined polygons and additional attributes
gdf_combined = grouped.agg({'geometry': unary_union, 'farea': 'first'}).reset_index()

# Create a GeoDataFrame from the combined polygons
gdf_combined = gpd.GeoDataFrame(gdf_combined, geometry='geometry')

# Sort the GeoDataFrame by the timestamp
gdf_sorted = gdf_combined.sort_values('t')

# Select rows with a timestamp of 12:00:00
gdf_1200 = gdf_sorted[gdf_sorted['t'].dt.time == datetime.time(0, 0, 0)]

# Calculate the change in "farea" for each FireName
#gdf_1200['farea_change'] = gdf_1200.groupby('FireName')['farea'].diff()
gdf_1200['farea_change'] = gdf_1200.groupby('fireid')['farea'].diff()
#gdf_1200['farea_change'] = gdf_1200.groupby('FireID')['farea'].diff()

gdf_1200['farea_change'] = gdf_1200['farea_change'].clip(lower=0)

# Set values of 'farea_change' to zero if they are less than 50 acre change
#gdf_1200.loc[gdf_1200['farea_change'] < 0.005, 'farea_change'] = 0

# Replace NaN values in "farea_change" with 0
gdf_1200['farea_change'] = gdf_1200['farea_change'].fillna(0)

###
# Calculate the percent change in "farea" for each FireName
# gdf_1200_pt['percent_farea_change'] = (gdf_1200_pt['farea_change'] / gdf_1200_pt['farea']) * 100

# # Set values of 'percent_farea_change' to zero if the corresponding 'farea_change' is zero
# gdf_1200_pt['percent_farea_change'] = np.where(gdf_1200_pt['farea_change'] == 0, 0, gdf_1200_pt['percent_farea_change'])

###


gdf_1200_pt = gdf_1200.copy()

gdf_1200_pt['geometry'] = gdf_1200_pt['geometry'].centroid
gdf_1200_pt = gpd.GeoDataFrame(gdf_1200_pt, geometry='geometry')

# Assuming the geometry column contains Point objects
# Replace 'geometry_column' with the actual column name in your dataframe
gdf_1200_pt['Latitude'] = gdf_1200_pt['geometry'].y
gdf_1200_pt['Longitude'] = gdf_1200_pt['geometry'].x

gdf_1200_pt['farea_change_shifted'] = gdf_1200_pt.groupby('fireid')['farea_change'].shift(-1)
#gdf_1200_pt['farea_change_shifted'] = gdf_1200_pt.groupby('FireID')['farea_change'].shift(-1)

gdf_1200_pt['farea_change_shifted'] = gdf_1200_pt['farea_change_shifted'].fillna(0)

gdf_1200_pt['percent_Increase'] = ((gdf_1200_pt['farea']+gdf_1200_pt['farea_change_shifted'])-gdf_1200_pt['farea'])/(gdf_1200_pt['farea'])

columns_to_join = ['fireid', 't', 'n_pixels','n_newpixel','meanfrp','fperim','duration']
gdf_1200_pt = pd.merge(gdf_1200_pt, gdf[columns_to_join], on=['fireid', 't'], how='left')
#columns_to_join = ['FireID', 't', 'n_pixels','n_newpixel','meanFRP','fperim','duration']# meanfrp
#gdf_1200_pt = pd.merge(gdf_1200_pt, gdf[columns_to_join], on=['FireID', 't'], how='left')

# Convert the timestamp column to datetime type
gdf_1200_pt['t'] = pd.to_datetime(gdf_1200_pt['t'])

#date_string = '20190101'
#date = datetime.strftime(date_string, "%Y%m%d")

#gdf_1200_pt = gdf_1200_pt[gdf_1200_pt['t'] >= date_string]

#FireNameList = gdf_1200_pt["FireName"].unique()
FireNameList = gdf_1200_pt["fireid"].unique()
#FireNameList = gdf_1200_pt["FireID"].unique()
print("done")


C:\Users\admin-magstadt\anaconda3\envs\ee\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


done


In [10]:
#gdf_1200_pt = gdf_1200_pt.iloc[:10]
print(len(gdf_1200_pt))

11230


In [11]:
import requests
import pandas as pd
import xarray as xr
import os
from io import BytesIO

#os.chdir('C:/Users/admin-magstadt/Desktop/GRIDMET_VIIRS_10_9_23/TEMP/')

import time


start_time = time.time()  # Capture the start time



url = ["http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pr_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmax_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmin_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_sph_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_srad_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_th_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmn_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmx_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vs_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_bi_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm100_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm1000_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_erc_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pdsi_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vpd_1979_CurrentYear_CONUS.nc",
"http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_z_1979_CurrentYear_CONUS.nc"]


VariableList = [
    'precipitation_amount',
    'daily_maximum_relative_humidity',
    'daily_minimum_relative_humidity',
    'daily_mean_specific_humidity',
    'daily_mean_shortwave_radiation_at_surface',
    'daily_mean_wind_direction',
    'daily_minimum_temperature', 
    'daily_maximum_temperature', 
    'daily_mean_wind_speed',
    'daily_mean_burning_index_g',
    'dead_fuel_moisture_100hr',
    'dead_fuel_moisture_1000hr',
    'daily_mean_energy_release_component-g',
    'daily_mean_palmer_drought_severity_index', 
    'daily_mean_vapor_pressure_deficit',
    'daily_mean_palmer_z_index'
]


for variable, current_url in zip(VariableList, url):
    climate_values = []  # List to store extracted values for the current variable

    # Loop through the dataframe rows
    for idx, row in gdf_1200_pt.iterrows():
        # Extract the latitude, longitude, and date for the current row
        latitude = row["Latitude"]
        longitude = row["Longitude"]
        start_date = pd.to_datetime(row['t'])

        # Set the request parameters
        params = {
            "var": variable,
            "latitude": latitude,
            "longitude": longitude,
            "time": start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
            "addLatLon": "true",
            "accept": "netcdf"
        }

        # Send the request
        response = requests.get(current_url, params=params, timeout=600)
        if response.status_code == 200:
            # Store the response content in a BytesIO object
            in_memory_file = BytesIO(response.content)

            # Load the netCDF data from the in-memory file using xarray
            ds = xr.open_dataset(in_memory_file,engine='scipy')

            # Extract the value and append to the list
            value_at_point = ds[variable].values[0]
            climate_values.append(value_at_point)
#             # If request is successful, save the data to a temporary netCDF file
#             #filename = r"C:/Users/admin-magstadt/Desktop/GRIDMET_VIIRS_10_9_23/TEMP/temp_data.nc"
#             from pathlib import Path

#             #path = Path(r"A:/TEMP/")
#             #filename = r"C:/Users/admin-magstadt/Desktop/GRIDMET_VIIRS_10_9_23/TEMP/temp_data2.nc"
#             filename = "A:\\TEMP\\tempdata2.nc"

#             with open(filename, "wb") as f:
#                 f.write(response.content)

#             # Load the netCDF data using xarray
#             ds = xr.open_dataset(filename)

#             # Extract the value and append to the list
#             value_at_point = ds[variable].values[0]
#             climate_values.append(value_at_point)
        else:
            print(f"Failed to retrieve data for index {idx} using variable {variable}. Status code:", response.status_code)
            climate_values.append(None)

        if (idx + 1) % 1000 == 0:
            print(idx + 1)
            print(variable)
            


    # Append the climate_values list as a new column to gdf_1200_pt
    gdf_1200_pt[variable] = climate_values
    
    csv_filename = f"C:\\Users\\admin-magstadt\\Desktop\\GRIDMET_VIIRS_10_9_23\\data\\gdf_1200_pt_2021_{variable}.csv"
    gdf_1200_pt.to_csv(csv_filename, index=False)
    print(f"Saved to {csv_filename}")


    
end_time = time.time()  # Capture the end time

print(f"Time taken for request: {end_time - start_time:.2f} seconds")




1000
precipitation_amount
2000
precipitation_amount
3000
precipitation_amount
4000
precipitation_amount
5000
precipitation_amount
6000
precipitation_amount
7000
precipitation_amount
8000
precipitation_amount
9000
precipitation_amount
10000
precipitation_amount
11000
precipitation_amount
Saved to C:\Users\admin-magstadt\Desktop\GRIDMET_VIIRS_10_9_23\data\gdf_1200_pt_2021_precipitation_amount.csv
1000
daily_maximum_relative_humidity
2000
daily_maximum_relative_humidity
3000
daily_maximum_relative_humidity
4000
daily_maximum_relative_humidity
5000
daily_maximum_relative_humidity
6000
daily_maximum_relative_humidity
7000
daily_maximum_relative_humidity
8000
daily_maximum_relative_humidity
9000
daily_maximum_relative_humidity
10000
daily_maximum_relative_humidity
11000
daily_maximum_relative_humidity
Saved to C:\Users\admin-magstadt\Desktop\GRIDMET_VIIRS_10_9_23\data\gdf_1200_pt_2021_daily_maximum_relative_humidity.csv
1000
daily_minimum_relative_humidity
2000
daily_minimum_relative_humidit

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
import pandas as pd
import os

# Directory where CSVs are stored
directory = "C:\\Users\\admin-magstadt\\Desktop\\GRIDMET_VIIRS_10_9_23\\data"

# List of variable names, as you provided above
VariableList = [
    'precipitation_amount',
    'daily_maximum_relative_humidity',
    'daily_minimum_relative_humidity',
    'daily_mean_specific_humidity',
    'daily_mean_shortwave_radiation_at_surface',
    'daily_mean_wind_direction',
    'daily_minimum_temperature', 
    'daily_maximum_temperature', 
    'daily_mean_wind_speed',
    'daily_mean_burning_index_g',
    'dead_fuel_moisture_100hr',
    'dead_fuel_moisture_1000hr',
    'daily_mean_energy_release_component-g',
    'daily_mean_palmer_drought_severity_index', 
    'daily_mean_vapor_pressure_deficit',
    'daily_mean_palmer_z_index'
]

# Initialize a list to hold dataframes
dfs = []

# Read each CSV into a dataframe and append to the list
for variable in VariableList:
    csv_filename = os.path.join(directory, f"gdf_1200_pt_2019_{variable}.csv")
    df = pd.read_csv(csv_filename)
    dfs.append(df)

# Merge all dataframes on common columns
final_df = dfs[0]
for df in dfs[1:]:
    final_df = final_df.merge(df, how='inner', on=list(df.columns.intersection(final_df.columns)))

final_df


In [ ]:
# Remove rows where any of the selected columns have a value of 32767
final_df = final_df[~(final_df[VariableList] == 32767).any(axis=1)]


In [ ]:
import pandas as pd

# Calculate statistics for each variable
stats = final_df[VariableList].describe().loc[['min', 'max', 'mean', 'std']]

# Display the table
stats


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Set the style
# sns.set_style("whitegrid")

# fixed_xlim = [0, 1000]  # example range

# # Loop through each variable and plot a density scatter plot
# for var in VariableList:
#     plt.figure(figsize=(10, 6))
    
#     # Set the xlim for the plot
#     plt.xlim(fixed_xlim)
    
#     # Using the hexbin function for density scatter plots
#     hb = plt.hexbin(final_df[var], final_df["percent_Increase"], gridsize=100, cmap="viridis", mincnt=0.01)
#     #hb = plt.hexbin(final_df[var], final_df["farea_change_shifted"], gridsize=100, cmap="viridis", mincnt=0.01)

#     plt.colorbar(hb, label="Counts")
    
#     plt.title(f"Density Scatter plot of {var} vs. farea_change_shifted")
#     plt.xlabel(var)
#     plt.ylabel("farea_change_shifted")
#     plt.show()



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the data (assuming it's in final_df DataFrame)
# final_df = ...

# 1. Data Preprocessing
features = VariableList + ['farea']
X = final_df[features].values
y = final_df['percent_Increase'].values

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# 2. Model Creation
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)  # Linear activation for regression
])

model.compile(optimizer='adam', loss='mse')

# 3. Training
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

# 4. Evaluation
mse_test = model.evaluate(X_test, y_test)
print(f"Test MSE: {mse_test:.4f}")

# Optionally: Visualize training progress
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Plotting the actual vs. predicted values
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('Actual farea_change_shifted')
plt.ylabel('Predicted farea_change_shifted')
plt.title('Actual vs. Predicted farea_change_shifted')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')  # Ideal 1:1 line
plt.show()


In [ ]:
#gdf_1200_pt.to_csv('C:\\Users\\admin-magstadt\\Desktop\\GRIDMET_VIIRS_10_9_23\\data\\Gridmet_2019_VIIRS.csv', index=False)


In [ ]:
# import requests
# import pandas as pd
# import xarray as xr
# import io  

# url = ["http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pr_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmax_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmin_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_sph_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_srad_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_th_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmn_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmx_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vs_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_bi_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm100_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm1000_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_erc_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pdsi_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vpd_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_z_1979_CurrentYear_CONUS.nc"]


# VariableList = [
#     'precipitation_amount',
#     'daily_maximum_relative_humidity',
#     'daily_minimum_relative_humidity',
#     'daily_mean_specific_humidity',
#     'daily_mean_shortwave_radiation_at_surface',
#     'daily_mean_wind_direction',
#     'daily_minimum_temperature', 
#     'daily_maximum_temperature', 
#     'daily_mean_wind_speed',
#     'daily_mean_burning_index_g',
#     'dead_fuel_moisture_100hr',
#     'dead_fuel_moisture_1000hr',
#     'daily_mean_energy_release_component-g',
#     'daily_mean_palmer_drought_severity_index', 
#     'daily_mean_vapor_pressure_deficit',
#     'daily_mean_palmer_z_index'
# ]


# for variable, current_url in zip(VariableList, url):
#     climate_values = []  # List to store extracted values for the current variable

#     # Loop through the dataframe rows
#     for idx, row in gdf_1200_pt.iterrows():
#         # Extract the latitude, longitude, and date for the current row
#         latitude = row["Latitude"]
#         longitude = row["Longitude"]
#         start_date = pd.to_datetime(row['t'])

#         # Set the request parameters
#         params = {
#             "var": variable,
#             "latitude": latitude,
#             "longitude": longitude,
#             "time": start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
#             "addLatLon": "true",
#             "accept": "netcdf"
#         }

#         # Send the request
#         response = requests.get(current_url, params=params, timeout=10)

#         if response.status_code == 200:
#             # If request is successful, save the data to a temporary netCDF file
#             filename = f"temp_data_{variable}.nc"
#             with open(filename, "wb") as f:
#                 f.write(response.content)

#             # Load the netCDF data using xarray
#             ds = xr.open_dataset(filename)

#             # Extract the value and append to the list
#             value_at_point = ds[variable].values[0]
#             climate_values.append(value_at_point)
#         else:
#             print(f"Failed to retrieve data for index {idx} using variable {variable}. Status code:", response.status_code)
#             climate_values.append(None)

#         if (idx + 1) % 1000 == 0:
#             print(idx + 1)
#             print(variable)

#     # Append the climate_values list as a new column to gdf_1200_pt
#     gdf_1200_pt[variable] = climate_values


In [ ]:
# import requests
# import pandas as pd
# import xarray as xr
# import io  

# url = ["http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pr_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmax_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_rmin_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_sph_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_srad_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_th_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmn_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_tmmx_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vs_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_bi_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm100_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_fm1000_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_erc_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_pdsi_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_vpd_1979_CurrentYear_CONUS.nc",
# "http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_met_z_1979_CurrentYear_CONUS.nc"]


# VariableList = [
#     'precipitation_amount',
#     'daily_maximum_relative_humidity',
#     'daily_minimum_relative_humidity',
#     'daily_mean_specific_humidity',
#     'daily_mean_shortwave_radiation_at_surface',
#     'daily_mean_wind_direction',
#     'daily_minimum_temperature', 
#     'daily_maximum_temperature', 
#     'daily_mean_wind_speed',
#     'daily_mean_burning_index_g',
#     'dead_fuel_moisture_100hr',
#     'dead_fuel_moisture_1000hr',
#     'daily_mean_energy_release_component-g',
#     'daily_mean_palmer_drought_severity_index', 
#     'daily_mean_vapor_pressure_deficit',
#     'daily_mean_palmer_z_index'
# ]


# for variable, current_url in zip(VariableList, url):
#     climate_values = []  # List to store extracted values for the current variable

#     # Loop through the dataframe rows
#     for idx, row in gdf_1200_pt.iterrows():
#         # ... [rest of the loop setup]

#         # Send the request
#         response = requests.get(current_url, params=params, timeout=10)

#         if response.status_code == 200:
#             # Write the content to an in-memory bytes buffer
#             buffer = io.BytesIO(response.content)
            
#             # Load the netCDF data using xarray directly from the buffer
#             ds = xr.open_dataset(buffer)

#             # Extract the value and append to the list
#             value_at_point = ds[variable].values[0]
#             climate_values.append(value_at_point)
#         else:
#             print(f"Failed to retrieve data for index {idx} using variable {variable}. Status code:", response.status_code)
#             climate_values.append(None)

#         if (idx + 1) % 5 == 0:
#             print(idx + 1)
#             print(variable)

#     # Append the climate_values list as a new column to gdf_1200_pt
#     gdf_1200_pt[variable] = climate_values